In [2]:
import sys
import os
sys.path.append('../..')
from src import getSecret

import requests

import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

# Replace with your Google email and password
googleUsername = getSecret('GOOGLE_USERNAME')
googlePassword = getSecret('GOOGLE_PASSWORD')

url = 'https://myactivity.google.com/product/youtube'

# Set up the web driver (replace with the path to your driver if needed)
driver = webdriver.Chrome(ChromeDriverManager().install())

# Navigate to the URL
driver.get(url)

# Log in to your Google account
email_input = driver.find_element("xpath",'//*[@id="identifierId"]')
email_input.send_keys(googleUsername)
email_input.send_keys(Keys.RETURN)
time.sleep(2)  # Wait for the password field to appear

password_input = driver.find_element("xpath",'//*[@id="password"]/div[1]/div/div[1]/input')
password_input.send_keys(googlePassword)
password_input.send_keys(Keys.RETURN)
time.sleep(5)  # Wait for the page to load

# Get the page's HTML content
html_content = driver.page_source

# Close the browser window
driver.quit()

# Process the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')
print(soup.prettify())


# html_content = '''
# PASTE YOUR HTML CONTENT HERE
# '''

# soup = BeautifulSoup(html_content, 'html.parser')

# videos = soup.find_all('div', class_='uUy2re')

# for video in videos:
#     title_element = video.find('a', class_='l8sGWb')
#     channel_element = video.find('a', target='_blank', rel='noopener noreferrer')
    
#     title = title_element.text
#     channel = channel_element.text
#     url = title_element['href']

#     print(f"Title: {title}\nChannel: {channel}\nURL: {url}\n")

/var/folders/8s/4d1fkp3s5fjcpzf1njmk1nvh0000gn/T/ipykernel_48671/4106977663.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="identifierId"]"}
  (Session info: chrome=112.0.5615.137)
Stacktrace:
0   chromedriver                        0x00000001012ad670 chromedriver + 4298352
1   chromedriver                        0x00000001012a5bbc chromedriver + 4266940
2   chromedriver                        0x0000000100ed8758 chromedriver + 280408
3   chromedriver                        0x0000000100f13b38 chromedriver + 523064
4   chromedriver                        0x0000000100f4c080 chromedriver + 753792
5   chromedriver                        0x0000000100f062d0 chromedriver + 467664
6   chromedriver                        0x0000000100f07354 chromedriver + 471892
7   chromedriver                        0x000000010126d6c4 chromedriver + 4036292
8   chromedriver                        0x0000000101271c64 chromedriver + 4054116
9   chromedriver                        0x00000001012782d8 chromedriver + 4080344
1

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()

email = getSecret('GOOGLE_USERNAME')
password = getSecret('GOOGLE_PASSWORD')

def Google_Sign_in():

    driver.get("https://accounts.google.com/signin/v2/identifier?flowName=GlifWebSignIn&flowEntry=ServiceLogin")
    
    ## Puts in email
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "identifierId"))).send_keys(email)

    driver.find_element("id","identifierNext").click()

    ##Puts in password
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.NAME, "password"))).send_keys(password)

    driver.find_element("id","passwordNext").click()

Google_Sign_in()

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000102ca9670 chromedriver + 4298352
1   chromedriver                        0x0000000102ca1bbc chromedriver + 4266940
2   chromedriver                        0x00000001028d4758 chromedriver + 280408
3   chromedriver                        0x000000010290fb38 chromedriver + 523064
4   chromedriver                        0x0000000102948080 chromedriver + 753792
5   chromedriver                        0x00000001029022d0 chromedriver + 467664
6   chromedriver                        0x0000000102903354 chromedriver + 471892
7   chromedriver                        0x0000000102c696c4 chromedriver + 4036292
8   chromedriver                        0x0000000102c6dc64 chromedriver + 4054116
9   chromedriver                        0x0000000102c742d8 chromedriver + 4080344
10  chromedriver                        0x0000000102c6e970 chromedriver + 4057456
11  chromedriver                        0x0000000102c458dc chro

In [6]:
import sys
import os
sys.path.append('../..')
from src import getSecret

import undetected_chromedriver as uc
import random,time,os,sys
from selenium.webdriver.common.keys import Keys

GMAIL = getSecret('GOOGLE_USERNAME')
PASSWORD = getSecret('GOOGLE_PASSWORD')

chrome_options = uc.ChromeOptions()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--profile-directory=Default")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--disable-plugins-discovery")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("user_agent=DN")
chrome_options.add_argument("--disable-web-security")
chrome_options.add_argument("--user-data-dir")
chrome_options.add_argument("--allow-running-insecure-content")

driver = uc.Chrome(options=chrome_options)

driver.delete_all_cookies()

driver.get("https://accounts.google.com/o/oauth2/v2/auth/oauthchooseaccount?redirect_uri=https%3A%2F%2Fdevelopers.google.com%2Foauthplayground&prompt=consent&response_type=code&client_id=407408718192.apps.googleusercontent.com&scope=email&access_type=offline&flowName=GeneralOAuthFlow")

driver.find_element("xpath","/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[1]/div/form/span/section/div/div/div[1]/div/div[1]/div/div[1]/input").send_keys(GMAIL)
driver.find_element("xpath","/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[1]/div/form/span/section/div/div/div[1]/div/div[1]/div/div[1]/input").send_keys(Keys.RETURN)

time.sleep(10)

KeyboardInterrupt: 

In [4]:
import sys
import os
sys.path.append('../..')
from src import getSecret

import undetected_chromedriver as uc
import random,time,os,sys
from selenium.webdriver.common.keys import Keys

GMAIL = getSecret('GOOGLE_USERNAME')
PASSWORD = getSecret('GOOGLE_PASSWORD')

chrome_options = uc.ChromeOptions()
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument("--disable-popup-blocking")
# chrome_options.add_argument("--profile-directory=Default")
# chrome_options.add_argument("--ignore-certificate-errors")
# chrome_options.add_argument("--disable-plugins-discovery")
# chrome_options.add_argument("--incognito")
# chrome_options.add_argument("user_agent=DN")
# chrome_options.add_argument("--disable-web-security")
# chrome_options.add_argument("--user-data-dir")
# chrome_options.add_argument("--allow-running-insecure-content")

driver = uc.Chrome(options=chrome_options)

driver.delete_all_cookies()

driver.get("https://stackoverflow.com/users/login")

google_login_button = driver.find_element('xpath','//button[@data-provider="google"]')
google_login_button.click()

time.sleep(2)

email_field = driver.find_element('xpath','//*[@id="identifierId"]')
email_field.send_keys(GMAIL)
email_field.send_keys(Keys.ENTER)

time.sleep(2)

password_field = driver.find_element('xpath','//*[@id="password"]/div[1]/div/div[1]/input')
password_field.send_keys(PASSWORD)
password_field.send_keys(Keys.ENTER)

time.sleep(5)

# wait for the 2 factor authenication 


driver.get("https://myactivity.google.com/product/youtube")

# The browser should now be logged in to Google and displaying the desired URL
# To close the browser, uncomment the following line
# driver.quit()


In [3]:
getSecret('GOOGLE_USERNAME')
getSecret('GOOGLE_PASSWORD')

'HZe3w4hcVM!pU7#Gp6LN'

In [11]:
import sys
import os
sys.path.append('../..')
from src import getSecret

import undetected_chromedriver as uc
import random,time,os,sys
from selenium.webdriver.common.keys import Keys

GMAIL = getSecret('GOOGLE_USERNAME')
PASSWORD = getSecret('GOOGLE_PASSWORD')

# Use the appropriate WebDriver for your browser
driver = webdriver.Chrome()
# driver = webdriver.Firefox()

# Navigate to the Stack Overflow login page
driver.get("https://stackoverflow.com/users/login")

# Click the "Log in with Google" button
google_login_button = driver.find_element("xpath",'//button[@data-provider="google"]')
google_login_button.click()

# Wait for the email field to appear
time.sleep(2)

# Fill in your Google email and press Enter
email_field = driver.find_element("xpath",'//*[@id="identifierId"]')
email_field.send_keys(GMAIL)
email_field.send_keys(Keys.ENTER)

# Wait for the password field to appear
time.sleep(2)

# Fill in your Google password and press Enter
password_field = driver.find_element("xpath",'//*[@id="password"]/div[1]/div/div[1]/input')
password_field.send_keys(PASSWORD)
password_field.send_keys(Keys.ENTER)

# Wait for the page to load
time.sleep(5)

# Navigate to the Google URL
driver.get("https://myactivity.google.com/product/youtube")

# The browser should now be logged in to Google and displaying the desired URL
# To close the browser, uncomment the following line
# driver.quit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="identifierId"]"}
  (Session info: chrome=112.0.5615.137)
Stacktrace:
0   chromedriver                        0x00000001011ad670 chromedriver + 4298352
1   chromedriver                        0x00000001011a5bbc chromedriver + 4266940
2   chromedriver                        0x0000000100dd8758 chromedriver + 280408
3   chromedriver                        0x0000000100e13b38 chromedriver + 523064
4   chromedriver                        0x0000000100e4c080 chromedriver + 753792
5   chromedriver                        0x0000000100e062d0 chromedriver + 467664
6   chromedriver                        0x0000000100e07354 chromedriver + 471892
7   chromedriver                        0x000000010116d6c4 chromedriver + 4036292
8   chromedriver                        0x0000000101171c64 chromedriver + 4054116
9   chromedriver                        0x00000001011782d8 chromedriver + 4080344
1

In [10]:
import sys
import os
sys.path.append('../..')

import google.auth
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Set up API client
def get_authenticated_service():
    # Replace 'client_secrets.json' with the name of your downloaded JSON client secrets file
    flow = InstalledAppFlow.from_client_secrets_file('../../secrets/oauth2.0_youtubeAPI_secretKey.json', ['https://www.googleapis.com/auth/youtube.readonly'])
    credentials = flow.run_local_server()
    return build('youtube', 'v3', credentials=credentials)

def get_watch_history(youtube):
    try:
        results = youtube.playlistItems().list(
            part="snippet",
            playlistId="HL",  # HL is the playlist ID for watch history
            maxResults=50  # You can set this value up to 50
        ).execute()

        for item in results["items"]:
            title = item["snippet"]["title"]
            video_id = item["snippet"]["resourceId"]["videoId"]
            print(f"{title}: https://www.youtube.com/watch?v={video_id}")

    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

# if __name__ == "__main__":
youtube_service = get_authenticated_service()
get_watch_history(youtube_service)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=243951296336-pdf56l49ibiholh08901tjbdaaceju63.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=dFDUfs71zpIZawdp5ijVg4fSVCrv2J&access_type=offline


KeyboardInterrupt: 